## Estruturas Criptográficas 2022/23
## TP3. Problema 1
### Grupo 7. Leonardo Berteotti e Paulo R. Pereira

Pretende-se a criação de um protótipo em Sagemath para o algoritmo KYBER: implementação de um KEM, que seja IND-CPA seguro, e um PKE que seja IND-CCA seguro.

Tal como apresentado [aqui](https://www.dropbox.com/sh/mx4bybl0d6e9g1m/AAAJsNOarzJ0fApTi8aBwzXVa/Kyber-20201001/Supporting_Documentation?dl=0&preview=kyber.pdf&subfolder_nav_tracking=1), o KYBER é IND-CCA2 seguro. 
Uma vez que o CCA2-KEM foi concebido para proporcionar um nível de segurança mais elevado do que o CCA-KEM, também pode ser considerado IND-CPA seguro. Isto deve-se ao facto de que qualquer ataque que quebre a segurança CPA, quebra também a segurança CCA2.

No entanto, o algoritmo de PKE conforme apresentado não é IND-CCA seguro, mas sim IND-CPA seguro. Portanto, para obter um PKE-IND-CCA, aplicar-se-á a transformação de Fujisaki-Okamoto, tal como está nos apontamentos da disciplina.

**Notação** 

$R$ corresponde ao anel $\mathbb{Z}[X]/(X^n+1)$ e $R_q$ ao anel $\mathbb{Z}_q[X]/(X^n+1)$ 

In [57]:
import random
import numpy as np
from cryptography.hazmat.primitives import hashes
from pickle import dumps, loads

**Clases auxiliares**

In [58]:
# Classe que implementa as multiplicações em R
# number-theoretic transform (NTT) 
class NTT:

    def __init__(self, n=128, q=None):
        
        if not  n in [32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs]) 
    
    def ntt(self,f):
        
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def invNtt(self,ff):                             
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
# Operações sobre matrizes e vetores
# Soma de matrizes
def sumMatrix(e1, e2, n):
    for i in range(len(e1)):
        e1[i] = sumVector(e1[i], e2[i], n)
    return e1

# Subtração de matrizes
def subMatrix(e1, e2, n):
    for i in range(len(e1)):
        e1[i] = subVector(e1[i], e2[i], n)
    return e1

# Multiplicação de matrizes
def multMatrix(vec1, vec2, n):
    for i in range(len(vec1)):
        vec1[i] = multVector(vec1[i], vec2[i],n)
    tmp = [0] * n
    for i in range(len(vec1)):
        tmp = sumVector(tmp, vec1[i], n)
    return tmp

# Multiplicação de uma matriz por um vector
def multMatrixVector(M, v, k, n) :
    for i in range(len(M)):
        for j in range(len(M[i])):
            M[i][j] = multVector(M[i][j], v[j], n)
    tmp = [[0] * n] * k 
    for i in range(len(M)):
        for j in range(len(M[i])):
            tmp[i] = sumVector(tmp[i], M[i][j],n)
    return tmp

# Soma de vetores
def sumVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] + ff2[i]))
    return res

# Multiplicação de vetores
def multVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] * ff2[i]))
    return res

# Subtração de vetores
def subVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] - ff2[i]))
    return res

#### **Implementação do esquema PKE IND-CPA seguro**

**KeyGen** (algoritmo 4)

São geradas a chave privada e a chave pública. O método começa por gerar a matriz $Â \in R_q^{k \times k}$ no domínio NTT e os vetores $s, e ∈ R_q$. Para tal foram utilizadas os métodos *Parse* e *CBD* implementados de acordo com os algoritmos 1 e 2, respetivamente.

**Encrypt** (algoritmo 5)

Cifra, com recurso à chave pública e um valor aleatório, uma mensagem *m* gerada a partir do anel $R_q$.

**Decrypt** (algoritmo 6)

TBC

In [72]:
class KYBER_PKE:
    
    def __init__(self, pset):
        self.n, self.q, self.T, self.k, self.n1, self.n2, self.du, self.dv, self.Rq = self.setup(pset)
    
    def setup(self, pset):
        n = 256
        q = 7681
        n2 = 2
        if pset == 512:
            k = 2
            n1 = 3
            du = 10
            dv = 4
        elif pset == 768:
            k = 3
            n1 = 2
            du = 10
            dv = 4
        elif pset == 1024:
            k = 4
            n1 = 2
            du = 11
            dv = 5
        else: print("Error: Parameter set not valid!")
            
        Zq.<w> = GF(q)[]
        fi = w^n + 1
        Rq.<w> = QuotientRing(Zq ,Zq.ideal(fi))
        
        T = NTT(n,q)
        
        return n,q,T,k,n1,n2,du,dv,Rq
    
    def bytes2Bits(self, byteArray):
        bitArray = []
        for elem in byteArray:
            bitElemArr = []
            for i in range(0,8): 
                bitElemArr.append(mod(elem//2**(mod(i,8)),2))
                for i in range(0,len(bitElemArr)):
                    bitArray.append(bitElemArr[i])
        return bitArray
    
    def G(self, h):
        digest = hashes.Hash(hashes.SHA3_512())
        digest.update(bytes(h))
        g = digest.finalize()
        return g[:32],g[32:]
    
    def XOF(self,b,b1,b2):
        digest = hashes.Hash(hashes.SHAKE128(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        digest.update(bytes(b2))
        m = digest.finalize()
        return m
    
    def PRF(self,b,b1): 
        digest = hashes.Hash(hashes.SHAKE256(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        return digest.finalize()

    def Compress(self,x,d) :
        coefficients = x.list()
        newCoefficients = []
        for c in coefficients:
            new = mod(round( int(2 ** d) / self.q * int(c)), int(2 ** d))
            newCoefficients.append(new)
        return self.Rq(newCoefficients)
    
    def Decompress(self,x,d) :
        coefficients = x.list()
        newCoefficients = []
        for c in coefficients:
            new = round(self.q / (2 ** d) * int(c))
            newCoefficients.append(new)
        return self.Rq(newCoefficients)
    
    # Algorithm 1
    def Parse(self, byteArray):
        i = 0
        j = 0
        a = []
        while j < self.n:
            d1 = byteArray[i] + 256 * mod(byteArray[i+1],16)
            d2 = byteArray[i+1]//16 + 16 * byteArray[i+2]
            if d1 < self.q :
                a.append(d1)
                j = j+1
            if d2 < self.q and j<self.n:
                a.append(d2)
                j = j+1
            i = i+3
        return self.Rq(a)
    
    
    # Algorithm 2
    def CBD(self, byteArray, nn):
        f=[0]*self.n
        bitArray = self.bytes2Bits(byteArray)
        for i in range(256):
            a = 0
            b = 0
            for j in range(nn):
                a += bitArray[2*i*nn + j]
                b += bitArray[2*i*nn + nn + j]
            f[i] = a-b
        return self.Rq(f)
    
    # Algorithm 3
    def Decode(self, byteArray, l):
        f = []
        bitArray = self.bytes2Bits(byteArray)
        for i in range(len(byteArray)):
            fi = 0
            for j in range(l):
                fi += int(bitArray[i*l+j]) * 2**j
            f.append(fi)
        return self.Rq(f)
    
    def Encode(self, f, l):
        # Convert polynomial f to a list of coefficients
        coefficients = list(f)
        # Create an empty list to store the bits
        bitArray = []
        # Iterate over the coefficients
        for fi in coefficients:
            # Convert the coefficient to l bits and append them to bitArray
            for j in range(l):
                bit = (int(fi) >> j) & 1
                bitArray.append(bit)
        # Convert the bit array to a bytearray
        byteList = []
        byte = 0
        byteLength = 0
        for bit in bitArray:
            byte += bit << byteLength
            byteLength += 1
            if byteLength == 8:
                byteList.append(byte)
                byte = 0
                byteLength = 0
        if byteLength > 0:
            byteList.append(byte)
        return bytes(byteList)


    
    # Algorithm 4
    def keyGen(self):
        d = bytearray(os.urandom(32))
        ro, sigma = self.G(d)
        N = 0
        
        # Generate matrix Â in Rq in NTT domain
        A = []
        for i in range(self.k):
            A.append([])
            for j in range(self.k):
                A[i].append(self.T.ntt(self.Parse(self.XOF(ro,j,i))))
        
        # Sample s in Rq from Bη1
        s = []  
        for i in range(self.k):
            s.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1
            
        # Sample e in Rq from Bη1
        e = []
        for i in range(self.k):
            e.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1

        for i in range(self.k) :
            s[i] = self.T.ntt(s[i])
            e[i] = self.T.ntt(e[i])
            
        t = sumMatrix(multMatrixVector(A,s,self.k,self.n), e, self.n)
        
        pk = t, ro
        sk = s
        
        return pk, sk
    
    # Algorithm 5
    def encrypt(self, pk, m , r):
        N = 0
        t, ro = pk
        
        # Generate matrix Â in Rq in NTT domain
        transposeA = []
        for i in range(self.k):
            transposeA.append([])
            for j in range(self.k):
                transposeA[i].append(self.T.ntt(self.Parse(self.XOF(ro,i,j))))
        
        # Sample r in Rq from Bη1
        rr = []
        for i in range(self.k):
            rr.insert(i,self.T.ntt(self.CBD(self.PRF(r, N), self.n1)))
            N += 1
        
        # Sample e1 in Rq from Bη2
        e1 = []
        for i in range(self.k):
            e1.insert(i,self.CBD(self.PRF(r, N), self.n2))
            N += 1
        
        # Sample e2 in Rq from Bη2
        e2 = self.CBD(self.PRF(r, N), self.n2)
        
        uAux = multMatrixVector(transposeA, rr, self.k, self.n)
        uAux2 = []
        for i in range(len(uAux)) :
            uAux2.append(self.T.invNtt(uAux[i]))
        uAux3 = sumMatrix(uAux2, e1, self.n)
        u = []
        for i in range(len(uAux3)) :
            u.append(self.Rq(uAux3[i]))
            
        vAux = multMatrix(t, rr, self.n)
        vAux1 = self.T.invNtt(vAux)
        vAux2 = self.Rq(sumVector(vAux1, e2, self.n))
        
        v = self.Rq(sumVector(vAux2, self.Decompress(m, 1), self.n))
        
        # Compress(u, du)
        c1 = []
        for i in range(len(u)):
            c1.append(self.Compress(u[i], self.du))
        
        # Compress(v, dv)
        c2 = self.Compress(v, self.dv)
        
        return c1, c2
    
    # Algorithm 6
    def decrypt(self, sk, c):
        c1, c2 = c
 
        u = []       
        for i in range(len(c1)):
            u.append(self.Decompress(c1[i], self.du))
        
        v = self.Decompress(c2,self.dv)

        s = sk
        
        uNTT = []
        for i in range(len(u)) :
            uNTT.append(self.T.ntt(u[i]))
        
        mAux = subVector(v, self.T.invNtt(multMatrix(s, uNTT, self.n)), self.n)

        m = self.Compress(self.Rq(mAux), 1)
        
        return m

#### **Implementação do KEM IND-CCA2-secure**

In [74]:
kyber = KYBER_PKE(1024)

public, private = kyber.keyGen()
m = "Hello there!"
x = kyber.Decode(m.encode(),1)
print(x)
print(kyber.Encode(x,1))
print("Original message:")

print(m)

ciphertext = kyber.encrypt(public, kyber.Decode(m.encode(),1), os.urandom(32))
print("\nCiphertext:")
print(ciphertext)

plaintext = kyber.decrypt(private, ciphertext)
print("\nDecrypted ciphertext:")
print(plaintext)

w^9
b'\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Original message:
Hello there!

Ciphertext:
([82*w^255 + 636*w^254 + 1264*w^253 + 1113*w^252 + 1979*w^251 + 131*w^250 + 1388*w^249 + 1959*w^248 + 943*w^247 + 930*w^246 + 1709*w^245 + 1959*w^244 + 230*w^243 + 546*w^242 + 1099*w^241 + 657*w^240 + 493*w^239 + 1466*w^238 + 1113*w^237 + 1650*w^236 + 402*w^235 + 1293*w^234 + 1476*w^233 + 1248*w^232 + 1195*w^231 + 1749*w^230 + 483*w^229 + 712*w^228 + 1627*w^227 + 1731*w^226 + 705*w^225 + 495*w^224 + 1020*w^223 + 83*w^222 + 1629*w^221 + 166*w^220 + 1094*w^219 + 1537*w^218 + 1729*w^217 + 1929*w^216 + 1552*w^215 + 1418*w^214 + 1546*w^213 + 840*w^212 + 2018*w^211 + 1850*w^210 + 511*w^209 + 1618*w^208 + 819*w^207 + 798*w^206 + 1519*w^205 + 1647*w^204 + 192*w^203 + 1082*w^202 + 261*w^201 + 950*w^200 + 1536*w^199 + 1020*w^198 + 1776*w^197 + 215*w^196 + 1250*w^195 + 290*w^194 + 1919*w^193 + 1410*w^192 + 629*w^191 + 4